##Autor: Filip Drvoderić <br>
##2. kolokvij - zadatak

In [ ]:
!pip install tensorflow

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

img_width, img_height = 150, 150
num_classes = 4

directories = ['/content/drive/MyDrive/Slike/Glioma', '/content/drive/MyDrive/Slike/Notumor', '/content/drive/MyDrive/Slike/Pituitary', '/content/drive/MyDrive/Slike/Meningioma']
images = []
labels = []

label_mappings = {'Glioma': 0, 'Notumor': 1, 'Pituitary': 2, 'Meningioma': 3}

index_to_class_name = {v: k for k, v in label_mappings.items()}

for dir_index, dir_path in enumerate(directories):
    for filename in os.listdir(dir_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(dir_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (img_width, img_height))
            images.append(image)
            labels.append(dir_index)

images = np.array(images).astype('float32') / 255.0
labels = to_categorical(labels, num_classes)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=25, validation_split=0.3, batch_size=32)

eval_result = model.evaluate(X_test, y_test)
print(f'Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}')

def make_prediction(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_width, img_height))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=[0, -1])
    prediction = model.predict(img)
    return prediction

image_path = '/content/drive/MyDrive/Slike/Testiranje/Te-piTr_0007.jpg'
prediction = make_prediction(image_path)
predicted_class_index = np.argmax(prediction, axis=1)
predicted_class_name = index_to_class_name[predicted_class_index[0]]
print(f'\n\nNa odabranoj slici je: {predicted_class_name}')

Epoch 1/25
1/1 [==============================] - 6s 6s/step - loss: 1.3959 - accuracy: 0.2381 - val_loss: 1.3845 - val_accuracy: 0.4000
Epoch 2/25
1/1 [==============================] - 2s 2s/step - loss: 1.2934 - accuracy: 0.4762 - val_loss: 1.3936 - val_accuracy: 0.3000
Epoch 3/25
1/1 [==============================] - 2s 2s/step - loss: 1.2301 - accuracy: 0.5714 - val_loss: 1.3884 - val_accuracy: 0.3000
Epoch 4/25
1/1 [==============================] - 2s 2s/step - loss: 1.1698 - accuracy: 0.4762 - val_loss: 1.3736 - val_accuracy: 0.4000
Epoch 5/25
1/1 [==============================] - 1s 1s/step - loss: 1.1353 - accuracy: 0.5714 - val_loss: 1.3547 - val_accuracy: 0.4000
Epoch 6/25
1/1 [==============================] - 1s 1s/step - loss: 1.0241 - accuracy: 0.7143 - val_loss: 1.3390 - val_accuracy: 0.4000
Epoch 7/25
1/1 [==============================] - 1s 1s/step - loss: 0.9848 - accuracy: 0.7143 - val_loss: 1.3279 - val_accuracy: 0.2000
Epoch 8/25
1/1 [=========================